In [6]:
import librosa
import numpy as np
from pydub import AudioSegment
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam
import tensorflow as tf 
import tensorflow_io as tfio
from sklearn.model_selection import train_test_split 
from keras.layers import Dense, Dropout, LSTM 

Extract MFCCs

In [7]:
def extractFeatures(filePath): 
    audio, sampleRate = librosa.load(filePath, res_type='kaiser_fast')  
    mfccs = librosa.feature.mfcc(y=audio, sr=sampleRate, n_mfcc=40) 
    mfccsProcessed = np.mean(mfccs.T,axis=0) 
    return mfccsProcessed 


Loading and labelling data

In [8]:

def loadAudioFiles(filePaths, label):
    features = []
    for file in filePaths:
        mfccsProcessed = extractFeatures(file)
        features.append(mfccsProcessed)
    return np.array(features), np.array([label] * len(features))

Importing files

In [9]:
correctFiles = ['audioFiles/adamKhaa .wav','audioFiles/dadKhaa.wav','audioFiles/hennaKhaa.wav','audioFiles/meKhaa.wav']
incorrectFiles = ['audioFiles/wrongKhaa1.wav','audioFiles/wrongKhaa2.wav','audioFiles/wrongKhaa3.wav']

In [12]:
correctFeatures, correctLabels = loadAudioFiles(correctFiles, 1)
incorrectFeatures, incorrectLabels = loadAudioFiles(incorrectFiles, 0)

Testing and Training

In [14]:
x = np.concatenate((correctFeatures, incorrectFeatures), axis=0)
y = np.concatenate((correctLabels, incorrectLabels), axis=0)

In [15]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [16]:
x_train = np.expand_dims(x_train, axis=2)
x_test = np.expand_dims(x_test, axis=2)

CNN Model

In [17]:
model = Sequential()
model.add(Dense(256, activation='relu', input_shape=(40,)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

In [18]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])

In [19]:
model.fit(x_train, y_train, epochs=50, batch_size=1, verbose=1, validation_data=(x_test, y_test))

Epoch 1/50
5/5 [==============================] - 1s 35ms/step - loss: 1.0370 - accuracy: 0.8000 - val_loss: 0.0041 - val_accuracy: 1.0000
Epoch 2/50
5/5 [==============================] - 0s 5ms/step - loss: 40.7809 - accuracy: 0.4000 - val_loss: 5.3708e-06 - val_accuracy: 1.0000
Epoch 3/50
5/5 [==============================] - 0s 7ms/step - loss: 44.2264 - accuracy: 0.4000 - val_loss: 5.2715e-08 - val_accuracy: 1.0000
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 29.5640 - accuracy: 0.2000 - val_loss: 8.1298e-10 - val_accuracy: 1.0000
Epoch 5/50
5/5 [==============================] - 0s 6ms/step - loss: 25.5432 - accuracy: 0.6000 - val_loss: 6.5188e-09 - val_accuracy: 1.0000
Epoch 6/50
5/5 [==============================] - 0s 6ms/step - loss: 27.4175 - accuracy: 0.6000 - val_loss: 6.9106e-07 - val_accuracy: 1.0000
Epoch 7/50
5/5 [==============================] - 0s 7ms/step - loss: 22.5511 - accuracy: 0.6000 - val_loss: 3.8391e-05 - val_accuracy: 1.0000
Epo

Evaluate model

In [20]:
test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=1)
print(f"Test accuracy: {test_accuracy}")

1/1 [==============================] - 0s 132ms/step - loss: 3.2108e-08 - accuracy: 1.0000
Test accuracy: 1.0
